In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Exploratory Data Analysis (EDA)

In [ ]:
import pandas as pd

df = pd.read_csv("/kaggle/input/bank-transaction-dataset-for-fraud-detection/bank_transactions_data_2.csv")
df.head().T

In [ ]:
df.info()

Before the initiation of data processing, this dataset contains **16** columns of attributes: <br>
(i) 2 columns of *float64* attributes, <br> (ii) 3 columns of *int64* attributes, and <br> (iii) 11 columns of *object* attributes.

In [ ]:
# Before Data Processing
print(f"Before Data Processing")
df.describe(include='all').T

In [ ]:
data_column_names = df.columns
print(f"The name of columns in this datasets are:")
print()
print(data_column_names)

In [ ]:
for col in data_column_names:
    print(f"Number of Unique values in the `{col}` attribute:" , df[col].nunique())
    print(f"Distinct unique values in the `{col}` attribute:" , df[col].unique())
    print("-" * 70)

There isn't any null columns or attributes discovered thus far.

In [ ]:
df.isnull().sum()

In [ ]:
# Missing values
missing = df.isnull().sum()
missing_percent = (missing / len(df)) * 100
missing_df = pd.DataFrame({
    'Missing Values': missing,
    'Percentage': missing_percent
})
missing_df

In [ ]:
# Duplicate rows
df.duplicated().sum()

# Data Processing

## 1. Conversion of Data Types

Update **TransactionDate** into *datetime*

In [ ]:
# Before Transformation 
print(f"Data Type before transformation:", df['TransactionDate'].dtypes)

# After Transformation
from datetime import datetime

df['TransactionDate'] = pd.to_datetime(df['TransactionDate'], format='ISO8601')
print(f"Data Type after transformation:", df['TransactionDate'].dtypes)

Update **PreviousTransactionDate** into *datetime*

In [ ]:
# Before Transformation 
print(f"Data Type before transformation:", df['PreviousTransactionDate'].dtypes)

# After Transformation
df['PreviousTransactionDate'] = pd.to_datetime(df['PreviousTransactionDate'], format='ISO8601')
print(f"Data Type after transformation:", df['PreviousTransactionDate'].dtypes)

In [ ]:
# Before Transformation 
print(f"Data Type before transformation:", df['TransactionAmount'].dtypes)

# After Transformation
## -- no change required

In [ ]:
# Before Transformation
print(f"Data Type before transformation:", df['CustomerAge'].dtypes)

# After Transformation
## -- no change required

In [ ]:
df.info()

In [ ]:
# After Data Processing
print(f"After Data Processing")
df.describe(include='all').T

In [ ]:
# Export DataFrame to CSV
df.to_csv('output.csv', index=True)

## 2. Data Columns Separation 
* numeric
* categorical

In [ ]:
# Column type separation
id_cols = ['TransactionID', 'AccountID']
date_cols = ['TransactionDate', 'PreviousTransactionDate']

# Convert to datetime
for col in date_cols:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')

# Identify numeric vs categorical
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()

print("Numeric columns:", numeric_cols)
print("Categorical columns:", categorical_cols)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

for col in numeric_cols:
    fig, axes = plt.subplots(1, 2, figsize=(12,4))
    sns.histplot(df[col], kde=True, bins=40, ax=axes[0], color="steelblue")
    axes[0].set_title(f"{col} Distribution")
    sns.boxplot(x=df[col], ax=axes[1], color="orange")
    axes[1].set_title(f"{col} Boxplot")
    plt.show()
    print(df[col].describe().T)
    print(f"Skewness: {df[col].skew():.2f}, Kurtosis: {df[col].kurt():.2f}\n")

# Univariate Analysis

In [ ]:
cat_cols = [
    'TransactionType', 
    'Location', 
    'Channel', 
    'CustomerOccupation'
]

for col in cat_cols:
    plt.figure(figsize=(12,5))
    order = df[col].value_counts().index
    sns.countplot(x=col, data=df, order=order, palette="viridis")
    plt.title(f"{col} Distribution")
    plt.xticks(rotation=45)
    plt.show()

# Bivariate Analysis
* Categorical vs Numerical 

In [ ]:
plt.figure(figsize=(10,5))
sns.boxplot(x='TransactionType', y='TransactionAmount', data=df)
plt.title("Transaction Amount by Transaction Type")
plt.show()

plt.figure(figsize=(10,5))
sns.boxplot(x='Channel', y='TransactionAmount', data=df)
plt.title("Transaction Amount by Channel")
plt.show()

plt.figure(figsize=(12,5))
sns.boxplot(x='CustomerOccupation', y='AccountBalance', data=df)
plt.title("Account Balance by Occupation")
plt.xticks(rotation=45)
plt.show()

# Archive 

KIV code, potentially problematic code

In [ ]:
duplicate = df.duplicated().sum()
duplicate_percent = (duplicate / len(df)) * 100
duplicate_df = pd.DataFrame({
    'Duplicate Values': duplicate,
    'Percentage': duplicate_percent
})

## 2. More about 1st Quartile of Transaction Amount

In [ ]:
from pandasql import sqldf
import pandas as pd

q1 = """
        SELECT TransactionDate, AVG(TransactionAmount) AS AVG_Trans_Amt
        from df
        WHERE 
        GROUP BY TransactionDate
        ORDER BY TransactionDate;
    """
# q2 = """SELECT * from df WHERE TransactionAmount BETWEEN 81.885 AND 211.14;"""
# q3 = """SELECT * from df WHERE TransactionAmount BETWEEN 211.14 AND 414.5275;"""
# q4 = """SELECT * from df WHERE TransactionAmount > 414.5275;"""

q1 = sqldf(q1, env=None)
q1.tail()

In [ ]:
# https://www.geeksforgeeks.org/python/time-series-plot-or-line-plot-with-pandas/

y = q1['TransactionDate']
x = q1['AVG_Trans_Amt']

# Create a dataframe using the two lists
Q1 = pd.DataFrame(
    { 'TransactionDate' : y , 'AVG_Trans_Amt' : x })

Q1

In [ ]:
# use plot() method on the dataframe

Q1.plot( 'TransactionDate' , 'AVG_Trans_Amt' )

plt.title("Average Transaction Amount")  # Chart title
plt.show()

In [ ]:
import plotly.express as px

a = px.line(Q1, x="TransactionDate", y="AVG_Trans_Amt", title='Average Transaction Amount')
a.show()

In [ ]:
sqldf(q2, env=None)

In [ ]:
print(sqldf('''SELECT * from df 
            WHERE TransactionAmount 
            BETWEEN 81.885 AND 211.14
            ORDER BY TransactionAmount;
         '''))

In [ ]:
sqldf(q2, env=None).info()

In [ ]:
print(sqldf('''SELECT * from df 
                WHERE TransactionAmount 
                BETWEEN 81.885 AND 211.14;
                '''))

In [ ]:
df.info()

In [ ]:
print(sqldf('''SELECT species, island 
FROM penguins 
LIMIT 5'''))